# 2. Follower Recommendations:

## Given a dataset representing a social network's following graph. Use MapReduce to recommend the users to follow for another users who do have a mutual followers, but do not follow each other.

#### Importing libraries:

In [1]:
from functools import reduce
from collections import defaultdict
import random

#### Generating friendships file:

In [2]:
def generate_friendship_graph(num_users, avg_connections):
    friendship_graph = {}

    for user in range(1, num_users + 1):
        num_connections = random.randint(1, 2 * avg_connections)
        connections = random.sample(range(1, num_users + 1), num_connections)
        try:
            connections.remove(user)  # Remove self-connection
        except:
            pass
        friendship_graph[user] = connections

    return friendship_graph

def save_friendship_graph_to_file(friendship_graph, filename):
    with open(filename, 'w') as file:
        for user, connections in friendship_graph.items():
            connections_str = ' '.join(map(str, connections))
            file.write(f"{user} {connections_str}\n")

num_users = 100 
avg_connections = 10 
output_filename = "follower_graph.txt"

graph = generate_friendship_graph(num_users, avg_connections)
save_friendship_graph_to_file(graph, output_filename)
print(f"Friendship graph with {num_users} users saved to {output_filename}.")

Friendship graph with 100 users saved to follower_graph.txt.


#### Reading a file

In [3]:
with open("follower_graph.txt","r")as file:
    text = file.readlines()

In [4]:
friends_data = []
for line in text:
    new_line = line.strip().split(" ")
    friends_data.append(new_line)   

#### Defining map functions:

In [5]:
def map_function_get_friends(line):
    person_friends = defaultdict(int)  
    person = line[0] 
    friends = line[1:]
    person_friends[person] = friends

    return person_friends

In [6]:
def map_function_get_friends_of_friends(person):
    
    person_friends_of_friends = defaultdict(list)
    
    for person_id, all_friends in person.items():
        person_id = str(person_id)
        friends_of_friends_list = []
    
        for friend in all_friends:
            friend_id = int(friend)-1
            friends_of_friends_list.extend(people[friend_id][friend])
    
    person_friends_of_friends[person_id] = friends_of_friends_list   
    
    return person_friends_of_friends.items()

#### Mapping data:

In [7]:
mapped_data = map(map_function_get_friends,friends_data)

#### Sorting values:

In [8]:
people = []

for person_friends_pairs in mapped_data:
    people.append(person_friends_pairs)

#### Mapping data:

In [9]:
mapped_data2 = map(map_function_get_friends_of_friends,people)

#### Sorting values:

In [10]:
intermediate_func = defaultdict(list)

for person_suggested_friends in mapped_data2:
    for person, recommended_friends in person_suggested_friends:
        
#         Removing main person from the recommended friends list:
        new_list = [i for i in recommended_friends if i != person] 
    
        intermediate_func[person].extend(new_list)   

#### Defining reduce function:

In [11]:
def reduce_function(map,full_list):
    
#     Converting list elements from strings to integers:
    int_list = [eval(i) for i in full_list]
    
#     Removing duplicates from the list:
    final_list = list(set(int_list))

    return map, sorted(final_list)

#### Reducing

In [12]:
friends_recommendations = []

for item in intermediate_func.items():
    result = reduce(reduce_function,item)
    friends_recommendations.append(result)

In [13]:
for friends_recommendation in friends_recommendations:
    print(f"Friends recommendations for person No. {friends_recommendation[0]}: {friends_recommendation[1]}.")

Friends recommendations for person No. 1: [5, 8, 11, 13, 18, 21, 23, 26, 29, 30, 36, 37, 39, 40, 41, 46, 49, 53, 57, 59, 68, 70, 73, 74, 77, 81, 85, 86, 87, 89, 94, 96, 99].
Friends recommendations for person No. 2: [1, 4, 6, 7, 8, 9, 11, 12, 13, 15, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 74, 76, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 99, 100].
Friends recommendations for person No. 3: [1, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 46, 47, 48, 49, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 69, 70, 73, 75, 76, 78, 79, 80, 81, 83, 84, 85, 87, 88, 89, 90, 92, 94, 96, 97, 98, 99, 100].
Friends recommendations for person No. 4: [1, 2, 3, 5, 6, 7, 8, 11, 12, 14, 15, 18, 19, 20, 21, 23, 25, 26, 27, 28, 